# This is Jeopardy!

#### Overview

This project is slightly different than others you have encountered thus far. Instead of a step-by-step tutorial, this project contains a series of open-ended requirements which describe the project you'll be building. There are many possible ways to correctly fulfill all of these requirements, and you should expect to use the internet, Codecademy, and/or other resources when you encounter a problem that you cannot easily solve.

#### Project Goals

You will work to write several functions that investigate a dataset of _Jeopardy!_ questions and answers. Filter the dataset for topics that you're interested in, compute the average difficulty of those questions, and train to become the next Jeopardy champion!

## Prerequisites

In order to complete this project, you should have completed the Pandas lessons in the <a href="https://www.codecademy.com/learn/paths/analyze-data-with-python">Analyze Data with Python Skill Path</a>. You can also find those lessons in the <a href="https://www.codecademy.com/learn/data-processing-pandas">Data Analysis with Pandas course</a> or the <a href="https://www.codecademy.com/learn/paths/data-science/">Data Scientist Career Path</a>.

Finally, the <a href="https://www.codecademy.com/learn/practical-data-cleaning">Practical Data Cleaning</a> course may also be helpful.

## Project Requirements

In [4]:
import pandas as pd
import re
pd.set_option('display.max_colwidth', None)

jeopardy_csv = pd.read_csv('jeopardy.csv')
#print(jeopardy_csv.head(10))
print(jeopardy_csv.columns.tolist())
jeopardy_csv.rename(columns={'Show Number': 'show_number', ' Air Date': 'air_date', ' Category': 'category', ' Round': 'round', 
                             ' Value':'value', 
                             ' Question': 'question', 
                             ' Answer':'answer'
}, inplace=True)
print(jeopardy_csv.head())

['Show Number', ' Air Date', ' Round', ' Category', ' Value', ' Question', ' Answer']
   show_number    air_date      round                         category value  \
0         4680  2004-12-31  Jeopardy!                          HISTORY  $200   
1         4680  2004-12-31  Jeopardy!  ESPN's TOP 10 ALL-TIME ATHLETES  $200   
2         4680  2004-12-31  Jeopardy!      EVERYBODY TALKS ABOUT IT...  $200   
3         4680  2004-12-31  Jeopardy!                 THE COMPANY LINE  $200   
4         4680  2004-12-31  Jeopardy!              EPITAPHS & TRIBUTES  $200   

                                                                                                      question  \
0             For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory   
1  No. 2: 1912 Olympian; football star at Carlisle Indian School; 6 MLB seasons with the Reds, Giants & Braves   
2                     The city of Yuma in this state has a record average of 4,055 hours of

2. Write a function that filters the dataset for questions that contains all of the words in a list of words. For example, when the list `["King", "England"]` was passed to our function, the function returned a DataFrame of 49 rows. Every row had the strings `"King"` and `"England"` somewhere in its `" Question"`.

   Test your function by printing out the column containing the question of each row of the dataset.

In [5]:
def filter(dataframe, words):
    filtered = dataframe
    for word in words: 
        filtered = filtered[filtered['question'].str.contains(word, case=False, na=False)]
    return filtered

filtered_df = filter(jeopardy_csv, ['King','England'])
print(filtered_df['question'])
            

4953                    Both England's King George V & FDR put their stamp of approval on this "King of Hobbies"
6337      In retaliation for Viking raids, this "Unready" king of England attacks Norse areas of the Isle of Man
9191                    This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt
11710               This Scotsman, the first Stuart king of England, was called "The Wisest Fool in Christendom"
13454                                       It's the number that followed the last king of England named William
                                                           ...                                                  
208295        In 1066 this great-great grandson of Rollo made what some call the last Viking invasion of England
208742                      Dutch-born king who ruled England jointly with Mary II & is a tasty New Zealand fish
213870                In 1781 William Herschel discovered Uranus & initially named it after this

3. Test your original function with a few different sets of words to try to find some ways your function breaks. Edit your function so it is more robust.

   For example, think about capitalization. We probably want to find questions that contain the word `"King"` or `"king"`.
   
   You may also want to check to make sure you don't find rows that contain substrings of your given words. For example, our function found a question that didn't contain the word `"king"`, however it did contain the word `"viking"` &mdash; it found the `"king"` inside `"viking"`. Note that this also comes with some drawbacks &mdash; you would no longer find questions that contained words like `"England's"`.

In [6]:
def robust_filter(dataframe, words, strict=True):
    words = [w.lower() for w in words]
    filtered = dataframe.copy()
    
    for word in words:
        if strict == True: 
            pattern = r'\b' + re.escape(word) + r'\b'
        else:
            pattern = word
        filtered = filtered[filtered['question'].str.lower().str.contains(pattern, na=False, regex=True)]
    return filtered

filtered_df = robust_filter(jeopardy_csv, ['King','England'])
print(filtered_df['question'])

4953                    Both England's King George V & FDR put their stamp of approval on this "King of Hobbies"
6337      In retaliation for Viking raids, this "Unready" king of England attacks Norse areas of the Isle of Man
9191                    This king of England beat the odds to trounce the French in the 1415 Battle of Agincourt
11710               This Scotsman, the first Stuart king of England, was called "The Wisest Fool in Christendom"
13454                                       It's the number that followed the last king of England named William
                                                           ...                                                  
201168                                  Popular Saint-Exupery character waiting around to become king of England
204778                                 This southern colony was named for a king of England, the II of that name
208742                      Dutch-born king who ruled England jointly with Mary II & is a tasty 

4. We may want to eventually compute aggregate statistics, like `.mean()` on the `" Value"` column. But right now, the values in that column are strings. Convert the`" Value"` column to floats. If you'd like to, you can create a new column with float values.

   While most of the values in the `" Value"` column represent a dollar amount as a string, note that some do not &mdash; these values will need to be handled differently!

   Now that you can filter the dataset of question, use your new column that contains the float values of each question to find the "difficulty" of certain topics. For example, what is the average value of questions that contain the word `"King"`?
   
   Make sure to use the dataset that contains the float values as the dataset you use in your filtering function.

In [7]:
clean_values = jeopardy_csv['value'].str.replace('$','').str.replace(',','')
jeopardy_csv['float_value'] = pd.to_numeric(clean_values, errors='coerce')
jeopardy_csv['float_value'].head()

king_df = robust_filter(jeopardy_csv, ['King'], strict=True)
mean_of_king_questions = king_df['float_value'].mean()
print(mean_of_king_questions)

817.6001483679526


5. Write a function that returns the count of unique answers to all of the questions in a dataset. For example, after filtering the entire dataset to only questions containing the word `"King"`, we could then find all of the unique answers to those questions. The answer "Henry VIII" appeared 55 times and was the most common answer.

In [8]:
def unique_answer(df, words):
    filtered_df = robust_filter(df, words)

    answers = filtered_df['answer']

    counts = answers.value_counts()

    return counts

unique_answer_counts = unique_answer(jeopardy_csv, ['king'])
print(unique_answer_counts)

    

answer
Henry VIII                           54
Solomon                              31
Louis XIV                            31
David                                29
Sweden                               24
                                     ..
Oscar                                 1
Henderson                             1
(Catherine) Howard                    1
"The Circle Of Life"                  1
a pyramid (the pyramids accepted)     1
Name: count, Length: 1499, dtype: int64


6. Explore from here! This is an incredibly rich dataset, and there are so many interesting things to discover. There are a few columns that we haven't even started looking at yet. Here are some ideas on ways to continue working with this data:

 * Investigate the ways in which questions change over time by filtering by the date. How many questions from the 90s use the word `"Computer"` compared to questions from the 2000s?
 * Is there a connection between the round and the category? Are you more likely to find certain categories, like `"Literature"` in Single Jeopardy or Double Jeopardy?
 * Build a system to quiz yourself. Grab random questions, and use the <a href="https://docs.python.org/3/library/functions.html#input">input</a> function to get a response from the user. Check to see if that response was right or wrong.

In [9]:
import random

def jeopardy_quiz(df):
    while True: 
        question_row = df.sample(n=1).iloc[0]

        question_text = question_row['question']
        question_answer = question_row['answer']

        print('\nQuestion:')
        print(question_text)

        user_answer = input('Your answer (or type quit to exit): ')

        if user_answer.lower().strip() == 'quit':
            print("Thanks for playing!")
            break 
        if user_answer.lower().strip() == str(question_answer).lower().strip():
            print("That was correct! :)") 
        else:
            print(f'Sorry, that is not correct, the correct answer is {question_answer}')

## Solution

7. Compare your program to our <a href="https://content.codecademy.com/PRO/independent-practice-projects/jeopardy/jeopardy_solution.zip">sample solution code</a> - remember, that your program might look different from ours (and probably will) and that's okay!

8. Great work! Visit <a href="https://discuss.codecademy.com/t/this-is-jeopardy-challenge-project-python-pandas/462365">our forums</a> to compare your project to our sample solution code. You can also learn how to host your own solution on GitHub so you can share it with other learners! Your solution might look different from ours, and that's okay! There are multiple ways to solve these projects, and you'll learn more by seeing others' code.

In [18]:
jeopardy_quiz(jeopardy_csv)


Question:
A trip to Italy in 1875 inspired his sculpture "The Age of Bronze"


Your answer (or type quit to exit):  Gallaleo


Sorry, that is not correct, the correct answer is Auguste Rodin

Question:
Part of speech represented in the last word of the phrase "L'etat c'est moi"


Your answer (or type quit to exit):  noun


Sorry, that is not correct, the correct answer is Pronoun

Question:
Around 1500 he sketched not only a flying machine, but a parachute


...

Sorry, that is not correct, the correct answer is Minute Maid

Question:
Who -- the Who; Where -- La Jolla Playhouse; When -- 1992; What -- this musical


Your answer (or type quit to exit):  Hevan


Sorry, that is not correct, the correct answer is Tommy

Question:
This word comes from Greek for good or easy death


Your answer (or type quit to exit):  quit


Thanks for playing!


In [16]:
jeopardy_csv['air_date'] = pd.to_datetime(jeopardy_csv['air_date'])

nineties = (jeopardy_csv['air_date'] >= '1990-01-10') & (jeopardy_csv['air_date'] < '2000-01-01')
two_thousands = (jeopardy_csv['air_date'] >= '2000-01-01') & (jeopardy_csv['air_date'] < '2010-01-01')

computer_90s = jeopardy_csv[nineties & jeopardy_csv['question'].str.contains('Computer', case=False, na=False)]
computer_00s = jeopardy_csv[two_thousands & jeopardy_csv['question'].str.contains('Computer', case=False, na=False)]

print('1990s:', len(computer_90s))
print('2000s:', len(computer_00s))


1990s: 98
2000s: 268


In [24]:
def number_of_mentions_by_date(df, word, start_year, end_year):
    df['air_date'] = pd.to_datetime(df['air_date'])
    mask = (df['air_date'] >= f'{start_year}-01-01') & (df['air_date'] < f'{end_year}-01-01')
    return len(df[mask & df['question'].str.contains(word, case=False, na=False)])

number_of_mentions_by_date(jeopardy_csv, 'Morocco', 1990, 2020)

46

In [33]:
jeopardy_csv['round'].unique()
jeopardy_csv['category'].unique()


round_by_category_counts = jeopardy_csv.groupby(['round','category']).size()
print(round_by_category_counts)


round             category               
Double Jeopardy!  "-ARES"                    5
                  "...OD" WORDS              5
                  "1", "2", "3"              5
                  "20" QUESTIONS             5
                  "A" + 4                    5
                                            ..
Jeopardy!         “NORTH”                    5
                  “STREETS”                  5
Tiebreaker        CHILD'S PLAY               1
                  LITERARY CHARACTERS        1
                  THE AMERICAN REVOLUTION    1
Length: 31686, dtype: int64


In [41]:
questions = jeopardy_csv[jeopardy_csv['question'].str.contains('word', na=False)]
print(questions.head())

     show_number   air_date             round           category   value  \
70          5957 2010-07-06         Jeopardy!     PARTS OF PEACH    $600   
103         5957 2010-07-06  Double Jeopardy!  IN THE DICTIONARY   $1200   
109         5957 2010-07-06  Double Jeopardy!  IN THE DICTIONARY  $5,000   
115         5957 2010-07-06  Double Jeopardy!  IN THE DICTIONARY   $2000   
121         3751 2000-12-18         Jeopardy!          FOREWORDS    $100   

                                                                                                               question  \
70                                                                       5-letter word for the hard interior of a peach   
103                                                                                      Maize is another word for this   
109                                                     This word for someone who walks comes from the Latin for "foot"   
115                                        Kayak is